In [38]:
import numpy as np
import pandas as pd
import sqlite3 as sq3
import os

In [50]:
base_dir = os.path.abspath(os.getcwd())
db_dir = os.path.abspath(os.path.join(base_dir, "../Data"))
db_path = os.path.join(db_dir, "oltp_311.db")
conn = sq3.connect(db_path)

print("Connection to oltp_311.db established.")

Connection to oltp_311.db established.


In [51]:
query = """
SELECT
    -- service_request (core transaction)
    sr.request_id,
    sr.created_timestamp,
    sr.closed_timestamp,
    sr.resolution_action_updated_timestamp,
    sr.status,
    sr.channel,
    sr.resolution_description,

    -- agency
    a.agency_code,
    a.agency_name,

    -- complaint
    c.complaint_type,
    c.descriptor AS complaint_descriptor,

    -- location
    l.board_id,
    l.zip,
    l.city,
    l.latitude,
    l.longitude,
    l.x_coordinate_state_plane,
    l.y_coordinate_state_plane,
    l.location_type,
    l.incident_address,
    l.street_name,
    l.cross_street_1,
    l.cross_street_2,
    l.intersection_street_1,
    l.intersection_street_2,
    l.landmark,
    l.bbl,

    -- borough
    b.borough_name,

    -- park
    p.park_facility_name,
    p.park_borough

FROM service_request sr
LEFT JOIN agency a
    ON sr.agency_id = a.agency_id
LEFT JOIN complaint c
    ON sr.complaint_id = c.complaint_id
LEFT JOIN location l
    ON sr.location_id = l.location_id
LEFT JOIN borough b
    ON l.borough_id = b.borough_id
LEFT JOIN park p
    ON sr.park_id = p.park_id;
"""

df = pd.read_sql_query(query, conn)


In [53]:
pd.set_option('display.max_columns', None)
df.head()

,request_id,created_timestamp,closed_timestamp,resolution_action_updated_timestamp,status,channel,resolution_description,agency_code,agency_name,complaint_type,complaint_descriptor,board_id,zip,city,latitude,longitude,x_coordinate_state_plane,y_coordinate_state_plane,location_type,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,landmark,bbl,borough_name,park_facility_name,park_borough
0,66003043,2025-08-31 22:54:21,2025-08-31 23:46:13,2025-08-31T23:46:19.000,Closed,ONLINE,The Police Department responded to the complai...,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,12,11432.0,JAMAICA,40.712068,-73.785244,1043787.0,198775.0,Street/Sidewalk,87-23 178 STREET,178 STREET,EDGERTON BOULEVARD,88 AVENUE,EDGERTON BOULEVARD,88 AVENUE,178 STREET,4099130023.0,QUEENS,Unspecified,Unspecified
1,66003046,2025-09-01 00:02:45,2025-09-01 00:30:26,2025-09-01T00:30:31.000,Closed,MOBILE,The Police Department responded to the complai...,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,08,11238.0,BROOKLYN,40.676174,-73.961713,994870.0,185627.0,Street/Sidewalk,435 PROSPECT PLACE,PROSPECT PLACE,GRAND AVENUE,CLASSON AVENUE,GRAND AVENUE,CLASSON AVENUE,PROSPECT PLACE,3011550085.0,BROOKLYN,Unspecified,Unspecified
2,66003048,2025-08-31 21:54:19,2025-08-31 22:17:42,2025-08-31T22:17:52.000,Closed,MOBILE,The Police Department issued a summons in resp...,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,13,11224.0,BROOKLYN,40.579741,-73.983210,988914.0,150492.0,Street/Sidewalk,2792 WEST 15 STREET,WEST 15 STREET,HART PLACE,NEPTUNE AVENUE,HART PLACE,NEPTUNE AVENUE,WEST 15 STREET,3069960096.0,BROOKLYN,Unspecified,Unspecified
3,66003052,2025-09-01 00:27:54,2025-09-01 01:07:23,2025-09-01T01:07:28.000,Closed,PHONE,The Police Department issued a summons in resp...,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,02,11205.0,BROOKLYN,40.693835,-73.977916,990374.0,192060.0,Residential Building/House,112 ST EDWARDS STREET,ST EDWARDS STREET,AUBURN PLACE,NORTH ELLIOTT WALK,AUBURN PLACE,NORTH ELLIOTT WALK,ST EDWARDS STREET,3020340134.0,BROOKLYN,Unspecified,Unspecified
4,66003056,2025-08-31 22:06:50,2025-08-31 22:46:36,2025-08-31T22:46:38.000,Closed,MOBILE,The Police Department responded to the complai...,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,04,11207.0,BROOKLYN,40.687134,-73.912676,1008468.0,189630.0,Street/Sidewalk,71 ELDERT STREET,ELDERT STREET,BUSHWICK AVENUE,EVERGREEN AVENUE,BUSHWICK AVENUE,EVERGREEN AVENUE,ELDERT STREET,3034090058.0,BROOKLYN,Unspecified,Unspecified


In [54]:
def conv_to_days(waittime: pd.Timedelta) -> float | None:  # must be a timedelta type
    """Convert wait times to days

    Args:
        waittime (pd.Timedelta): Time difference between closing and created date

    Returns:
        float: If the waittime is a valid time delta, then return the time in days
        None: Returned when waittime is NaT
    """
    try:
        days = abs(waittime.components[0])
        hours = waittime.components[1] / 24
        mins = waittime.components[2] / 60 / 24
        secs = waittime.components[3] / 60 / 60 / 24
        if waittime.components[0] < 0:
            return (days + hours + mins + secs) * -1
        else:
            return days + hours + mins + secs
    except:
        # if NaT, return None
        pass
    
    
df["created_timestamp"] = pd.to_datetime(df["created_timestamp"])
df["closed_timestamp"] = pd.to_datetime(df["closed_timestamp"])
df["waittime"] = df["closed_timestamp"] - df["created_timestamp"]
df["waittime"] = df["waittime"].map(conv_to_days)

In [55]:
# Keep rows with non-negative wait times only
df = df[df['waittime'] >= 0]
df.shape

(599932, 31)

In [56]:
df["waittime"].describe(percentiles=[0.99, 0.999, 0.9999, 0.99999, 0.999999])

count       599932.000000
mean             0.098853
std              0.201573
min              0.000069
99%              0.650317
99.9%            1.664995
99.99%           5.279911
99.999%          9.076572
99.9999%        45.767052
max             87.477442
Name: waittime, dtype: float64

In [57]:
print(f"{df.shape[0]} Rows, {df.shape[1]} Columns")
na_counts = df.isna().sum()
print(f"{'Column':<35} {'NA Counts':<12} {'NA %':<5}")
for col in na_counts.index:
    if na_counts[col] == 0: continue
    print(f"{col:<35} {na_counts[col]:<12} {na_counts[col]/df.shape[0]:.2%}")

599932 Rows, 31 Columns
Column                              NA Counts    NA % 
resolution_description              1            0.00%
complaint_descriptor                18335        3.06%
zip                                 2960         0.49%
city                                26816        4.47%
latitude                            2947         0.49%
longitude                           2947         0.49%
x_coordinate_state_plane            2947         0.49%
y_coordinate_state_plane            2946         0.49%
incident_address                    2951         0.49%
street_name                         2953         0.49%
cross_street_1                      4105         0.68%
cross_street_2                      3708         0.62%
intersection_street_1               4105         0.68%
intersection_street_2               3708         0.62%
landmark                            26822        4.47%
bbl                                 47422        7.90%
park_facility_name                  851  

In [62]:
CAT_COLS = [
    "agency_code",
    "agency_name",
    "complaint_type",
    "complaint_descriptor",
    "location_type",
    "board_id",
    "borough_name",
    "channel",
    "park_facility_name",
    "park_borough",
    # "incident_address",
    # "street_name",
    # "cross_street_1",
    # "cross_street_2",
    # "intersection_street_1",
    # "intersection_street_2",
    "city",
    # "landmark",
]

for i, col in enumerate(CAT_COLS):
    print(f"Processing {col}... ({i+1}/{len(CAT_COLS)})")
    val_cts = df[col].value_counts()
    unique, counts = val_cts.index, val_cts.values

    for i in range(len(unique)):
        key, count = unique[i], counts[i]
        new_key = key.upper() if count > 100 or key.upper() == "UNSPECIFIED" else "OTHER"
        df.loc[df[col] == key, col] = new_key
print()

for col in CAT_COLS:
    val_cts = df[col].value_counts()
    unique, counts = val_cts.index, val_cts.values
    if len(unique) > 10: continue

    print(f"Column: {col}")
    print(f"{'Value':<35} {'Count':<7} {'Percentage %':<15}")
    for i in range(len(unique)):
        key, count = unique[i], counts[i]
        print(f"{unique[i]:<35} {counts[i]:<7} {counts[i]/(np.sum(list(counts)).item()):.2%}")

    print()
    print("="*70)

Processing agency_code... (1/11)
Processing agency_name... (2/11)
Processing complaint_type... (3/11)
Processing complaint_descriptor... (4/11)
Processing location_type... (5/11)
Processing board_id... (6/11)
Processing borough_name... (7/11)
Processing channel... (8/11)
Processing park_facility_name... (9/11)
Processing park_borough... (10/11)
Processing city... (11/11)

Column: agency_code
Value                               Count   Percentage %   
NYPD                                599932  100.00%

Column: agency_name
Value                               Count   Percentage %   
NEW YORK CITY POLICE DEPARTMENT     599932  100.00%

Column: borough_name
Value                               Count   Percentage %   
BROOKLYN                            173876  28.98%
BRONX                               161353  26.90%
QUEENS                              158139  26.36%
MANHATTAN                           91407   15.24%
STATEN ISLAND                       15142   2.52%
UNSPECIFIED             

In [64]:
df["created_timestamp"] = pd.to_datetime(df["created_timestamp"])

df["created_year"] = df["created_timestamp"].dt.year
df["created_month"] = df["created_timestamp"].dt.month
df["created_day"] = df["created_timestamp"].dt.day
df["created_hour"] = df["created_timestamp"].dt.hour
df["created_weekday"] = df["created_timestamp"].dt.weekday

df["closed_timestamp"] = pd.to_datetime(df["closed_timestamp"])

df["closed_year"] = df["closed_timestamp"].dt.year
df["closed_month"] = df["closed_timestamp"].dt.month
df["closed_day"] = df["closed_timestamp"].dt.day
df["closed_hour"] = df["closed_timestamp"].dt.hour
df["closed_weekday"] = df["closed_timestamp"].dt.weekday
df.head()

,request_id,created_timestamp,closed_timestamp,resolution_action_updated_timestamp,status,channel,resolution_description,agency_code,agency_name,complaint_type,complaint_descriptor,board_id,zip,city,latitude,longitude,x_coordinate_state_plane,y_coordinate_state_plane,location_type,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,landmark,bbl,borough_name,park_facility_name,park_borough,waittime,created_year,created_month,created_day,created_hour,created_weekday,closed_year,closed_month,closed_day,closed_hour,closed_weekday
0,66003043,2025-08-31 22:54:21,2025-08-31 23:46:13,2025-08-31T23:46:19.000,Closed,ONLINE,The Police Department responded to the complai...,NYPD,NEW YORK CITY POLICE DEPARTMENT,ILLEGAL PARKING,PAPER LICENSE PLATES,12,11432.0,JAMAICA,40.712068,-73.785244,1043787.0,198775.0,STREET/SIDEWALK,87-23 178 STREET,178 STREET,EDGERTON BOULEVARD,88 AVENUE,EDGERTON BOULEVARD,88 AVENUE,178 STREET,4099130023.0,QUEENS,UNSPECIFIED,UNSPECIFIED,0.036019,2025,8,31,22,6,2025,8,31,23,6
1,66003046,2025-09-01 00:02:45,2025-09-01 00:30:26,2025-09-01T00:30:31.000,Closed,MOBILE,The Police Department responded to the complai...,NYPD,NEW YORK CITY POLICE DEPARTMENT,ILLEGAL PARKING,PAPER LICENSE PLATES,08,11238.0,BROOKLYN,40.676174,-73.961713,994870.0,185627.0,STREET/SIDEWALK,435 PROSPECT PLACE,PROSPECT PLACE,GRAND AVENUE,CLASSON AVENUE,GRAND AVENUE,CLASSON AVENUE,PROSPECT PLACE,3011550085.0,BROOKLYN,UNSPECIFIED,UNSPECIFIED,0.019225,2025,9,1,0,0,2025,9,1,0,0
2,66003048,2025-08-31 21:54:19,2025-08-31 22:17:42,2025-08-31T22:17:52.000,Closed,MOBILE,The Police Department issued a summons in resp...,NYPD,NEW YORK CITY POLICE DEPARTMENT,ILLEGAL PARKING,PAPER LICENSE PLATES,13,11224.0,BROOKLYN,40.579741,-73.983210,988914.0,150492.0,STREET/SIDEWALK,2792 WEST 15 STREET,WEST 15 STREET,HART PLACE,NEPTUNE AVENUE,HART PLACE,NEPTUNE AVENUE,WEST 15 STREET,3069960096.0,BROOKLYN,UNSPECIFIED,UNSPECIFIED,0.016238,2025,8,31,21,6,2025,8,31,22,6
3,66003052,2025-09-01 00:27:54,2025-09-01 01:07:23,2025-09-01T01:07:28.000,Closed,PHONE,The Police Department issued a summons in resp...,NYPD,NEW YORK CITY POLICE DEPARTMENT,ILLEGAL PARKING,PAPER LICENSE PLATES,02,11205.0,BROOKLYN,40.693835,-73.977916,990374.0,192060.0,RESIDENTIAL BUILDING/HOUSE,OTHER,ST EDWARDS STREET,AUBURN PLACE,NORTH ELLIOTT WALK,AUBURN PLACE,NORTH ELLIOTT WALK,ST EDWARDS STREET,3020340134.0,BROOKLYN,UNSPECIFIED,UNSPECIFIED,0.027419,2025,9,1,0,0,2025,9,1,1,0
4,66003056,2025-08-31 22:06:50,2025-08-31 22:46:36,2025-08-31T22:46:38.000,Closed,MOBILE,The Police Department responded to the complai...,NYPD,NEW YORK CITY POLICE DEPARTMENT,ILLEGAL PARKING,PAPER LICENSE PLATES,04,11207.0,BROOKLYN,40.687134,-73.912676,1008468.0,189630.0,STREET/SIDEWALK,71 ELDERT STREET,ELDERT STREET,BUSHWICK AVENUE,EVERGREEN AVENUE,BUSHWICK AVENUE,EVERGREEN AVENUE,ELDERT STREET,3034090058.0,BROOKLYN,UNSPECIFIED,UNSPECIFIED,0.027616,2025,8,31,22,6,2025,8,31,22,6


In [65]:
df["has_precise_location"] = df["latitude"].notna()

In [69]:
import os
df_clean = df.copy()
base_dir = os.path.abspath(os.getcwd())
data_dir = os.path.abspath(os.path.join(base_dir, "../Data"))

os.makedirs(data_dir, exist_ok=True)

output_path = os.path.join(data_dir, "requests_cleaned.csv")
df_clean.to_csv(output_path, index=False)

print("Saved to:", output_path)

Saved to: /Users/nithin/ACADEMICS/DataScience/DSC413/NYC-311-Analysis/Data/requests_cleaned.csv
